This notebook implements the assembly of the libre-solar OSH project:

Currently includes only the frame of the enclosure


In [ ]:
# The following should reload if_lib if we change it on disk
%load_ext autoreload
%aimport if_lib, if_utils, if_dpp, if_graphics
%autoreload 1
import os
import json
import random

from if_utils import get_filename, show_data, save_traces

from if_lib import generate_random_challenge, read_HMAC, read_keypair, get_id_person, get_location_id, \
get_unit_id, get_resource_spec_id, get_resource, get_process, create_event, make_transfer, reduce_resource

from if_dpp import trace_query, check_traces, er_before, get_ddp

from if_graphics import vis_dpp, make_sankey, consol_trace

In [ ]:
USE_CASE = 'libre-solar'

# What endpoint are we talking to?
# debug
ENDPOINT = 'http://65.109.11.42:10000/api'
# ENDPOINT = 'http://zenflows-debug.interfacer.dyne.org'
# staging
# ENDPOINT = 'http://65.109.11.42:8000/api'
# ENDPOINT = 'https://zenflows-staging.interfacer.dyne.org/api'
# testing
# ENDPOINT = 'http://65.109.11.42:9000/api'
# ENDPOINT = 'https://zenflows-test.interfacer.dyne.org/api'

USERS_FILE = get_filename('cred_users.json', ENDPOINT, USE_CASE)
LOCS_FILE = get_filename('loc_users.json', ENDPOINT, USE_CASE)
UNITS_FILE = get_filename('units_data.json', ENDPOINT, USE_CASE)
SPECS_FILE = get_filename('res_spec_data.json', ENDPOINT, USE_CASE)

In [ ]:
# Read or define user data that is going to be used in the GraphQL calls

# create data structure to hold processes
process_data = {}

# create data structures to hold resources and events (possibly to compare results from track and trace)
res_data = {}
event_seq = []


if os.path.isfile(USERS_FILE):
    with open(USERS_FILE,'r') as f:
        users_data = json.loads(f.read())
    print("Credentials file available for users")
else:
    users_data = {}
    users_data['one'] = {
      "userChallenges": {
        "whereParentsMet": "London",
        "nameFirstPet": "Fuffy",
        "nameFirstTeacher": "Jim",
        "whereHomeTown": "Paris",
        "nameMotherMaid": "Wright"
      },
      "name": "User One",
      "username": "user1_username",
      "email": "user1@example.org",
      "note": "me.user1.org"
    }
    users_data['two'] = {
        "userChallenges": {
            "whereParentsMet":"Amsterdam",
            "nameFirstPet":"Toby",
            "nameFirstTeacher":"Juliet",
            "whereHomeTown":"Rome",
            "nameMotherMaid":"Banks"
        },
        "name": "User Two",
        "username": "user2",
        "email": "user2@example.org",
        "note" : "me.user2.org"
    }

    with open(USERS_FILE,'w') as f:
        json.dump(users_data, f)


if os.path.isfile(LOCS_FILE):
    with open(LOCS_FILE,'r') as f:
        locs_data = json.loads(f.read())
    print("Location file available")
else:
    locs_data = {}
    locs_data['one'] = {
        "name": "FabCity Hamburg",
        "lat" : 53.54651094994547,
        "long": 9.996357255816038,
        "addr": "Zippelhaus 5A, 20457 Hamburg, Germany",
        "note": "location.user1.org"
    }
    locs_data['two'] = {
        "name": "Fab City Hamburg",
        "lat" : 53.54651094994547,
        "long" : 9.996357255816038,
        "addr" : "Zippelhaus 5A, 20457 Hamburg, Germany",
        "note": "location.user2.org"
    }
    with open(LOCS_FILE,'w') as f:
        json.dump(locs_data, f)


if os.path.isfile(UNITS_FILE):
    with open(UNITS_FILE,'r') as f:
        units_data = json.loads(f.read())
    print(f"Unit file available")
else:
    units_data = {}
#     with open(file,'w') as f:
#         json.dump(units_data, f)



if os.path.isfile(SPECS_FILE):
    with open(SPECS_FILE,'r') as f:
        res_spec_data = json.loads(f.read())
    print(f"Resource Spec file available")
else:
    res_spec_data = {}
  

In [ ]:
# Read HMAC or get it from the server
read_HMAC(USERS_FILE, users_data, 'one', endpoint=ENDPOINT)
read_HMAC(USERS_FILE, users_data, 'two', endpoint=ENDPOINT)

In [ ]:
# Read the keypair

read_keypair(USERS_FILE, users_data, 'one')
read_keypair(USERS_FILE, users_data, 'two')

In [ ]:
# read or get id of the person
get_id_person(USERS_FILE, users_data, 'one', endpoint=ENDPOINT)
get_id_person(USERS_FILE, users_data, 'two', endpoint=ENDPOINT)

In [ ]:
# Read of get the location id
get_location_id(LOCS_FILE, users_data['one'], locs_data, 'one', endpoint=ENDPOINT)
get_location_id(LOCS_FILE, users_data['two'], locs_data, 'two', endpoint=ENDPOINT)

In [ ]:
# Get the ids of all units
get_unit_id(UNITS_FILE, users_data['two'], units_data, 'piece', 'u_piece', 'om2:one', endpoint=ENDPOINT)
get_unit_id(UNITS_FILE, users_data['two'], units_data, 'mass', 'kg', 'om2:kilogram', endpoint=ENDPOINT)
get_unit_id(UNITS_FILE, users_data['two'], units_data, 'volume', 'lt', 'om2:litre', endpoint=ENDPOINT)
get_unit_id(UNITS_FILE, users_data['one'], units_data, 'time', 'h', 'om2:hour', endpoint=ENDPOINT)


In [ ]:
# RESOURCE SPECIFICATIONS
# Read all the resource specifications
name = 'Allen_Key'
note = 'Specification for tool for assembly'
classification = 'A resource specification URI'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'M6nutplate'
note = 'Specification for screw'
classification = 'A resource specification URI'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'M6x12'
note = 'Specification for screw'
classification = 'A resource specification URI'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'M6x25cyl'
note = 'Specification for screw'
classification = 'A resource specification URI'
default_unit_id = units_data['mass']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'RAConnector'
note = 'Specification for connector'
classification = 'A resource specification URI'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'Tslot16'
note = 'Specification for a frame slot'
classification = 'A resource specification URI'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'Tslot32'
note = 'Specification for a frame slot'
classification = 'A resource specification URI'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'Tslot32con'
note = 'Specification for a frame slot'
classification = 'A resource specification URI'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'assembly_construction'
note = 'Specification for Frame assembly work'
classification = 'A resource specification URI'
default_unit_id = units_data['time']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'upper_frame'
note = 'Specification for upperframe'
classification = 'A resource specification URI'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'lower_frame'
note = 'Specification for lowerframe'
classification = 'A resource specification URI'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'pillar'
note = 'Specification for pillar'
classification = 'A resource specification URI'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)


# **** OLD isogown use case Read all the resource specifications
# TODO remove
name = 'soap'
note = 'Specification for soap to be used to wash the gowns'
classification = 'https://www.wikidata.org/wiki/Q34396'
default_unit_id = units_data['mass']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'water'
note = 'Specification for water to be used to wash the gowns'
classification = 'https://www.wikidata.org/wiki/Q283'
default_unit_id = units_data['volume']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'cotton'
note = 'Specification for cotton to be used to sew the gowns'
classification = 'https://www.wikidata.org/wiki/Q11457'
default_unit_id = units_data['mass']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'gown'
note = 'Specification for gowns'
classification = 'https://www.wikidata.org/wiki/Q89990310'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'surgical_operation'
note = 'Specification for surgical operations'
classification = 'https://www.wikidata.org/wiki/Q600236'
default_unit_id = units_data['time']['id']
get_resource_spec_id(SPECS_FILE, users_data['two'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

In [ ]:
# RESOURCES
# We create the resources that will not be saved to file as it is assumed they are recreated at each run

res_name = 'Allen_Key'
amount = 1

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
res_name = 'M6nutplate'
amount = 50

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
res_name = 'M6x12'
amount = 50

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
res_name = 'M6x25cyl'
amount = 50

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
res_name = 'RAConnector'
amount = 50

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
res_name = 'Tslot16'
amount = 50

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
res_name = 'Tslot32'
amount = 50

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
res_name = 'Tslot32con'
amount = 50

get_resource(res_data, res_spec_data, res_name, users_data['two'], event_seq, amount, endpoint=ENDPOINT)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the upper frame
# print(process_data)
process_name = 'UpperFrame'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the upper frame
# print(process_data)
process_name = 'UpperFrame_1'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the upper frame
# print(process_data)
process_name = 'UpperFrame_2'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the upper frame
# print(process_data)
process_name = 'UpperFrame_3'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the upper frame
# print(process_data)
process_name = 'UpperFrame_4'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the upper frame
# print(process_data)
process_name = 'LowerFrame'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the upper frame
# print(process_data)
process_name = 'LowerFrame_1'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the upper frame
# print(process_data)
process_name = 'LowerFrame_2'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the upper frame
# print(process_data)
process_name = 'LowerFrame_3'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the upper frame
# print(process_data)
process_name = 'LowerFrame_4'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Pillar
# print(process_data)
process_name = 'Pillar'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Pillar
# print(process_data)
process_name = 'Pillar_1'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Pillar
# print(process_data)
process_name = 'Pillar_2'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Pillar
# print(process_data)
process_name = 'First_Pillar_1'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Pillar
# print(process_data)
process_name = 'First_Pillar_2'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Pillar
# print(process_data)
process_name = 'Second_Pillar_1'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Pillar
# print(process_data)
process_name = 'Second_Pillar_2'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Pillar
# print(process_data)
process_name = 'Third_Pillar_1'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Pillar
# print(process_data)
process_name = 'Third_Pillar_2'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Pillar
# print(process_data)
process_name = 'Forth_Pillar_1'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# PROCESSES
# Create the process that wraps the assembly of the Pillar
# print(process_data)
process_name = 'Forth_Pillar_2'
user_data = users_data['one']
note = f"Assembly process {process_name} defined by {user_data['name']}"

get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)
# print(process_data)

In [ ]:
# UpperFrame 1
# Input event 1
# Define input consume Tslot32con event for UpperFrame Assembly: 
action = 'consume'
event_note='Input consume Tslot32con event for Upper Frame Assembly'
amount = 1
cur_pros = process_data['UpperFrame_1']
cur_res = res_data['Tslot32con_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 1
# Input event 2
# Define input consume Tslot16 event for UpperFrame Assembly: 
action = 'consume'
event_note='Input consume Tslot16 event for Upper Frame Assembly'
amount = 1
cur_pros = process_data['UpperFrame_1']
cur_res = res_data['Tslot16_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 1
# Input event 3
# Define input consume M6x25cyl event for UpperFrame Assembly: 
action = 'consume'
event_note='Input consume M6x25cyl event for Upper Frame Assembly'
amount = 1
cur_pros = process_data['UpperFrame_1']
cur_res = res_data['M6x25cyl_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 1
# Input event 4
# Define input use Allen Key for UpperFrame Assembly: 
action = 'use'
event_note='Input use Allen Key event for Upper Frame Assembly'
amount = 1
cur_pros = process_data['UpperFrame_1']
cur_res = res_data['Allen_Key_res']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 1
# Input event 5
# Define input event work with using Allen Key to assemble UpperFrame_1
action = 'work'
event_note='Make a corner junction between aTslot32con and Tslot16 by a M6x25cyl screw.'
amount = 0.05
cur_pros = process_data['UpperFrame_1']

effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.05


event_id, ts = create_event(users_data['one'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# UpperFrame 1
# Output event 1
# Define output event produce for UpperFrame
action = 'produce'
event_note='produce UpperFrame'
amount = 1
cur_pros = process_data['UpperFrame_1']

res_data['upper_frame_res'] = {
    "res_ref_id": f'upper_frame-{random.randint(0, 10000)}',
    "name": 'upper_frame',
    "spec_id": res_spec_data['upper_frame']['id']
}
cur_res = res_data['upper_frame_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 2
# Input event 1
# Define input event accept for UpperFrame 1
action = 'accept'
event_note='accept UpperFrame to be modified'
amount = 1
cur_pros = process_data['UpperFrame_2']
cur_res = res_data['upper_frame_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 2
# Input event 2
# Define input consume 2 x M6nutplates event for UpperFrame Assembly: 
action = 'consume'
event_note='Input consume 2 x M6nutplate event for Upper Frame Assembly'
amount = 2
cur_pros = process_data['UpperFrame_2']
cur_res = res_data['M6nutplate_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 2
# Input event 3
# Define input event work inserting 2 x M6nutplates into the upper and the outer slot of the Tslot16 beam of the Upper Frame
action = 'work'
event_note='Insert two M6nutplates into the upper and the outer slot of the Tslot16 beam of the Upper Frame.'
# amount = 0.05
cur_pros = process_data['UpperFrame_2']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.05

event_id, ts = create_event(users_data['one'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# UpperFrame 2
# Output event 1
# Define output modify Upper Frame
action = 'modify'
event_note='modify UpperFrame_2'
amount = 1
cur_pros = process_data['UpperFrame_2']
cur_res = res_data['upper_frame_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 3
# Input event 1
# Define input event accept for UpperFrame 2
action = 'accept'
event_note='accept UpperFrame to be modified'
amount = 1
cur_pros = process_data['UpperFrame_3']
cur_res = res_data['upper_frame_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 3
# Input event 2
# Define input consume 1 x M6nutplate event for UpperFrame Assembly
action = 'consume'
event_note='Input consume 1 x M6nutplate event for Upper Frame Assembly'
amount = 1
cur_pros = process_data['UpperFrame_3']
cur_res = res_data['M6nutplate_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 3
# Input event 3
# Define input consume 1 x Tslot16 event for UpperFrame Assembly
action = 'consume'
event_note='Input consume 1 x Tslot16 event for Upper Frame Assembly'
amount = 1
cur_pros = process_data['UpperFrame_3']
cur_res = res_data['Tslot16_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
# event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# UpperFrame 3
# Input event 4
# Define input consume 1 x M6x25cyl event for UpperFrame Assembly
action = 'consume'
event_note='Input consume 1 x M6x25cyl event for Upper Frame Assembly'
amount = 1
cur_pros = process_data['UpperFrame_3']
cur_res = res_data['M6x25cyl_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 3
# Input event 5
# Define input use Allen Key for UpperFrame Assembly: 
action = 'use'
event_note='Input use Allen Key event for Upper Frame Assembly'
amount = 1
cur_pros = process_data['UpperFrame_3']
cur_res = res_data['Allen_Key_res']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 3
# Input event 6
# Define input event work of the Upper Frame
action = 'work'
event_note='Before connecting the second Tslot16 beam you must insert a nutplate into the outer slot of the long Tslot32con, which will be enclosed by the M6x25cyl during the corner connection.'
# amount = 0.05
cur_pros = process_data['UpperFrame_3']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.05

event_id, ts = create_event(users_data['one'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# UpperFrame 3
# Output event 1
# Define output modify Upper Frame
action = 'modify'
event_note='modify UpperFrame_3'
amount = 1
cur_pros = process_data['UpperFrame_3']
cur_res = res_data['upper_frame_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 4
# Input event 1
# Define input event accept for UpperFrame 2
action = 'accept'
event_note='accept UpperFrame to be modified'
amount = 1
cur_pros = process_data['UpperFrame_4']
cur_res = res_data['upper_frame_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 4
# Input event 2
# Define input consume 3 x M6nutplate event for UpperFrame Assembly
action = 'consume'
event_note='Input consume 3 x M6nutplate event for Upper Frame Assembly'
amount = 3
cur_pros = process_data['UpperFrame_4']
cur_res = res_data['M6nutplate_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# EVENTS
# UpperFrame 4
# Input event 3
# Define input consume Tslot32con event for UpperFrame Assembly: 
action = 'consume'
event_note='Input consume Tslot32con event for Upper Frame Assembly'
amount = 2
cur_pros = process_data['UpperFrame_4']
cur_res = res_data['Tslot32con_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 4
# Input event 4
# Define input consume M6x25cyl event for UpperFrame Assembly: 
action = 'consume'
event_note='Input consume M6x25cyl event for Upper Frame Assembly'
amount = 2
cur_pros = process_data['UpperFrame_4']
cur_res = res_data['M6x25cyl_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 4
# Input event 5
# Define input use Allen Key for UpperFrame Assembly: 
action = 'use'
event_note='Input use Allen Key event for Upper Frame Assembly'
amount = 1
cur_pros = process_data['UpperFrame_4']
cur_res = res_data['Allen_Key_res']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# UpperFrame 4
# Input event 6
# Define input use Allen Key for UpperFrame Assembly: 
action = 'use'
event_note='Input use Allen Key event for Upper Frame Assembly'
amount = 1
cur_pros = process_data['UpperFrame_4']
cur_res = res_data['Allen_Key_res']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# UpperFrame 4
# Output event 1
# Define output modify Upper Frame
action = 'modify'
event_note='modify UpperFrame_4'
amount = 1
cur_pros = process_data['UpperFrame_4']
cur_res = res_data['upper_frame_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# EVENT
# LowerFrame 1
# Input event 1
# Define input consume Tslot32con event for LowerFrame Assembly: 
action = 'consume'
event_note='Input consume Tslot32con event for Lower Frame Assembly'
amount = 1
cur_pros = process_data['LowerFrame_1']
cur_res = res_data['Tslot32con_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 1
# Input event 2
# Define input consume Tslot16 event for LowerFrame Assembly: 
action = 'consume'
event_note='Input consume Tslot16 event for Lower Frame Assembly'
amount = 1
cur_pros = process_data['LowerFrame_1']
cur_res = res_data['Tslot16_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 1
# Input event 3
# Define input consume M6x25cyl event for LowerFrame Assembly: 
action = 'consume'
event_note='Input consume M6x25cyl event for Lower Frame Assembly'
amount = 1
cur_pros = process_data['LowerFrame_1']
cur_res = res_data['M6x25cyl_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 1
# Input event 4
# Define input use Allen Key for LowerFrame Assembly: 
action = 'use'
event_note='Input use Allen Key event for Lower Frame Assembly'
amount = 1
cur_pros = process_data['LowerFrame_1']
cur_res = res_data['Allen_Key_res']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 1
# Input event 5
# Define input event work of the Lower Frame
action = 'work'
event_note='Make a corner junction between aTslot32con and Tslot16 by a M6x25cyl screw.'
# amount = 0.05
cur_pros = process_data['LowerFrame_1']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.05

event_id, ts = create_event(users_data['one'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# LowerFrame 1
# Output event 1
# Define output event produce for LowerFrame
action = 'produce'
event_note='produce LowerFrame'
amount = 1
cur_pros = process_data['LowerFrame_1']

res_data['lower_frame_res'] = {
    "res_ref_id": f'lower_frame-{random.randint(0, 10000)}',
    "name": 'lower_frame',
    "spec_id": res_spec_data['lower_frame']['id']
}
cur_res = res_data['lower_frame_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 2
# Input event 1
# Define input event accept for LowerFrame 2
action = 'accept'
event_note='accept LowerFrame to be modified'
amount = 1
cur_pros = process_data['LowerFrame_2']
cur_res = res_data['lower_frame_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 2
# Input event 2
# Define input consume 2 x M6nutplates event for LowerFrame Assembly: 
action = 'consume'
event_note='Input consume 2 x M6nutplate event for Lower Frame Assembly'
amount = 4
cur_pros = process_data['LowerFrame_2']
cur_res = res_data['M6nutplate_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 2
# Input event 3
# Define input event work of the Lower Frame
action = 'work'
event_note='Insert therefore two M6nutplates into the upper slot and one into the lower slot underneath and one into the outer slot as usual '
# amount = 0.05
cur_pros = process_data['LowerFrame_2']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.05

event_id, ts = create_event(users_data['one'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# LowerFrame 2
# Output event 1
# Define output modify Lower Frame
action = 'modify'
event_note='modify LowerFrame_2'
amount = 1
cur_pros = process_data['LowerFrame_2']
cur_res = res_data['lower_frame_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 3
# Input event 1
# Define input event accept for Lower Frame 3
action = 'accept'
event_note='accept LowerFrame to be modified'
amount = 1
cur_pros = process_data['LowerFrame_3']
cur_res = res_data['lower_frame_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 3
# Input event 2
# Define input consume 1 x M6nutplates event for LowerFrame Assembly: 
action = 'consume'
event_note='Input consume 2 x M6nutplate event for Lower Frame Assembly'
amount = 1
cur_pros = process_data['LowerFrame_3']
cur_res = res_data['M6nutplate_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# EVENT
# LowerFrame 3
# Input event 3
# Define input consume Tslot16 event for LowerFrame Assembly: 
action = 'consume'
event_note='Input consume Tslot16con event for Lower Frame Assembly'
amount = 1
cur_pros = process_data['LowerFrame_3']
cur_res = res_data['Tslot16_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 3
# Input event 4
# Define input consume M6x25cyl event for LowerFrame Assembly: 
action = 'consume'
event_note='Input consume M6x25cyl event for Lower Frame Assembly'
amount = 1
cur_pros = process_data['LowerFrame_3']
cur_res = res_data['M6x25cyl_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 3
# Input event 5
# Define input use Allen Key for LowerFrame Assembly: 
action = 'use'
event_note='Input use Allen Key event for Lower Frame Assembly'
amount = 1
cur_pros = process_data['LowerFrame_3']
cur_res = res_data['Allen_Key_res']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 3
# Input event 6
# Define input event work of the Lower Frame
action = 'work'
event_note='Before connecting the second Tslot16 beam you must insert a nutplate into the outer slot of the long Tslot32con, which will be enclosed by the M6x25cyl during the corner connection.'
# amount = 0.05
cur_pros = process_data['LowerFrame_3']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.05

event_id, ts = create_event(users_data['one'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# LowerFrame 3
# Output event 1
# Define output modify Lower Frame
action = 'modify'
event_note='modify LowerFrame_3'
amount = 1
cur_pros = process_data['LowerFrame_3']
cur_res = res_data['lower_frame_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 4
# Input event 1
# Define input event accept for Lower Frame 3
action = 'accept'
event_note='accept LowerFrame to be modified'
amount = 1
cur_pros = process_data['LowerFrame_4']
cur_res = res_data['lower_frame_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 4
# Input event 2
# Define input consume 4 x M6nutplates event for LowerFrame Assembly: 
action = 'consume'
event_note='Input consume 4 x M6nutplate event for Lower Frame Assembly'
amount = 4
cur_pros = process_data['LowerFrame_4']
cur_res = res_data['M6nutplate_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# EVENT
# LowerFrame 4
# Input event 3
# Define input consume Tslot32con event for LowerFrame Assembly: 
action = 'consume'
event_note='Input consume Tslot32con event for Lower Frame Assembly'
amount = 1
cur_pros = process_data['LowerFrame_4']
cur_res = res_data['Tslot32con_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 4
# Input event 4
# Define input consume M6x25cyl event for LowerFrame Assembly: 
action = 'consume'
event_note='Input consume M6x25cyl event for Lower Frame Assembly'
amount = 2
cur_pros = process_data['LowerFrame_4']
cur_res = res_data['M6x25cyl_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 4
# Input event 5
# Define input use Allen Key for LowerFrame Assembly: 
action = 'use'
event_note='Input use Allen Key event for Lower Frame Assembly'
amount = 1
cur_pros = process_data['LowerFrame_4']
cur_res = res_data['Allen_Key_res']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# LowerFrame 4
# Input event 6
# Define input event work of the Lower Frame
action = 'work'
event_note='Before connecting the second Tslot16 beam you must insert a nutplate into the outer slot of the long Tslot32con, which will be enclosed by the M6x25cyl during the corner connection.'
# amount = 0.05
cur_pros = process_data['LowerFrame_4']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.05

event_id, ts = create_event(users_data['one'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# LowerFrame 4
# Output event 1
# Define output modify Lower Frame
action = 'modify'
event_note='modify LowerFrame_4'
amount = 1
cur_pros = process_data['LowerFrame_4']
cur_res = res_data['lower_frame_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# First Pillar
# Pillar_1
# Input event 1
# Define input consume Tslot32 event for Pillar Assembly: 
action = 'consume'
event_note='Input consume Tslot32con event for Pillar Assembly'
amount = 1
cur_pros = process_data['Pillar_1']
cur_res = res_data['Tslot32_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# First Pillar
# Pillar_1
# Input event 2
# Define input consume M6nutplate event for Pillar Assembly: 
action = 'consume'
event_note='Input consume 1 x M6nutplate event for Pillar Assembly'
amount = 1
cur_pros = process_data['Pillar_1']
cur_res = res_data['M6nutplate_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# First Pillar
# Pillar_1
# Input event 3
# Define input consume RAConnector event for Pillar Assembly: 
action = 'consume'
event_note='Input consume 1 x RAConnector event for Pillar Assembly'
amount = 1
cur_pros = process_data['Pillar_1']
cur_res = res_data['RAConnector_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# First Pillar
# Pillar_1
# Input event 4
# Define input consume 1 x M6x12 event for Pillar Assembly: 
action = 'consume'
event_note='Input consume 1 x M6x12 event for Pillar Assembly'
amount = 1
cur_pros = process_data['Pillar_1']
cur_res = res_data['RAConnector_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# First Pillar
# Pillar_1
# Input event 5
# Define input use Allen Key for LowerFrame Assembly: 
action = 'use'
event_note='Input use Allen Key event for Lower Frame Assembly'
amount = 1
cur_pros = process_data['Pillar_1']
cur_res = res_data['Allen_Key_res']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# First Pillar
# Pillar_1
# Input event 6
# Define input event work of the Pillar Pillar_1
action = 'work'
event_note='Insert one M6nutplate at one end of a Tslot32 beam and use it to connect one of the RAconnectors to that beam with a M6x12 screw.'
# amount = 0.05
cur_pros = process_data['Pillar_1']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.05

event_id, ts = create_event(users_data['one'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# First Pillar
# Pillar_1
# Output event 1
# Define output produce event of the Pillar
action = 'produce'
event_note='produce Pillar'
amount = 1
produced_resource = 'pillar'

cur_pros = process_data['Pillar_1']

res_data[f'{produced_resource}_res'] = {
    "res_ref_id": f'{produced_resource}-{random.randint(0, 10000)}',
    "name": f'{produced_resource}',
    "spec_id": res_spec_data[f'{produced_resource}']['id']
}
cur_res = res_data[f'{produced_resource}_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# First Pillar
# Pillar_2
# Input event 1
# Define input event accept of the Pillar
action = 'accept'
event_note='accept Pillar to be modified'
amount = 1
cur_pros = process_data['Pillar_2']
cur_res = res_data[f'{produced_resource}_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# First Pillar
# Pillar_2
# Input event 2
# Define input consume 3 x M6nutplate event for Pillar Assembly: 
action = 'consume'
event_note='Input consume 3 x M6nutplate event for Pillar Assembly'
amount = 3
cur_pros = process_data['Pillar_2']
cur_res = res_data['M6nutplate_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# First Pillar
# Pillar_2
# Input event 3
# Define input consume RAConnector event for Pillar Assembly: 
action = 'consume'
event_note='Input consume 1 x RAConnector event for Pillar Assembly'
amount = 1
cur_pros = process_data['Pillar_2']
cur_res = res_data['RAConnector_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# First Pillar
# Pillar_2
# Input event 4
# Define input use Allen Key for Pillar Assembly: 
action = 'use'
event_note='Input use Allen Key event for Lower Frame Assembly'
amount = 1
cur_pros = process_data['Pillar_2']
cur_res = res_data['Allen_Key_res']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# First Pillar
# Pillar_2
# Input event 5
# Define input event work of the Pillar Pillar_2
action = 'work'
event_note='Insert three M6nutplates more at the other end of the beam and use one of them to connect another RAconnector there.'
# amount = 0.05
cur_pros = process_data['Pillar_2']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.05

event_id, ts = create_event(users_data['one'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# First Pillar
# Pillar_2
# Output event 1
# Define output event modify Pillar
action = 'modify'
event_note='modify Pillar'
amount = 1
cur_pros = process_data['Pillar_2']
cur_res = res_data[f'{produced_resource}_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar
# Pillar_1
# Input event 1
# Define input consume Tslot32 event for Pillar Assembly: 
process_name = 'Second_Pillar_1'
action = 'consume'
event_note='Input consume Tslot32con event for Pillar Assembly'
amount = 1
cur_pros = process_data[f'{process_name}']
cur_res = res_data['Tslot32_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar
# Pillar_1
# Input event 2
# Define input consume M6nutplate event for Pillar Assembly: 
process_name = 'Second_Pillar_1'
action = 'consume'
event_note='Input consume 1 x M6nutplate event for Pillar Assembly'
amount = 1
cur_pros = process_data[f'{process_name}']
cur_res = res_data['M6nutplate_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar
# Pillar_1
# Input event 3
# Define input consume RAConnector event for Pillar Assembly: 
process_name = 'Second_Pillar_1'
action = 'consume'
event_note='Input consume 1 x RAConnector event for Pillar Assembly'
amount = 1
cur_pros = process_data[f'{process_name}']
cur_res = res_data['RAConnector_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar
# Pillar_1
# Input event 4
# Define input consume 1 x M6x12 event for Pillar Assembly: 
process_name = 'Second_Pillar_1'
action = 'consume'
event_note='Input consume 1 x M6x12 event for Pillar Assembly'
amount = 1
cur_pros = process_data[f'{process_name}']
cur_res = res_data['RAConnector_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar
# Pillar_1
# Input event 5
# Define input use Allen Key for LowerFrame Assembly: 
process_name = 'Second_Pillar_1'
action = 'use'
event_note='Input use Allen Key event for Lower Frame Assembly'
amount = 1
cur_pros = process_data[f'{process_name}']
cur_res = res_data['Allen_Key_res']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar
# Pillar_1
# Input event 6
# Define input event work of the Pillar Pillar_1
process_name = 'Second_Pillar_1'
action = 'work'
event_note='Insert one M6nutplate at one end of a Tslot32 beam and use it to connect one of the RAconnectors to that beam with a M6x12 screw.'
# amount = 0.05
cur_pros = process_data[f'{process_name}']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.05

event_id, ts = create_event(users_data['one'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# Second Pillar
# Pillar_1
# Output event 1
# Define output produce event of the Pillar
process_name = 'Second_Pillar_1'
action = 'produce'
event_note='produce Pillar'
amount = 1

produced_resource ='pillar'

cur_pros = process_data[f'{process_name}']

res_data[f'{produced_resource}_res'] = {
    "res_ref_id": f'{produced_resource}-{random.randint(0, 10000)}',
    "name": f'{produced_resource}',
    "spec_id": res_spec_data[f'{produced_resource}']['id']
}
cur_res = res_data[f'{produced_resource}_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar
# Pillar_2
# Input event 1
# Define input event accept of the Pillar
process_name = 'Second_Pillar_2'
action = 'accept'
event_note='accept Pillar to be modified'
amount = 1
cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{produced_resource}_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar
# Pillar_2
# Input event 2
# Define input consume 3 x M6nutplate event for Pillar Assembly: 
process_name = 'Second_Pillar_2'
action = 'consume'
event_note='Input consume 3 x M6nutplate event for Pillar Assembly'
amount = 3
cur_pros = process_data[f'{process_name}']
cur_res = res_data['M6nutplate_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar
# Pillar_2
# Input event 3
# Define input consume RAConnector event for Pillar Assembly: 
process_name = 'Second_Pillar_2'
action = 'consume'
event_note='Input consume 1 x RAConnector event for Pillar Assembly'
amount = 1
cur_pros = process_data[f'{process_name}']
cur_res = res_data['RAConnector_res']

event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                            res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar
# Pillar_2
# Input event 4
# Define input use Allen Key for Pillar Assembly: 
process_name = 'Second_Pillar_2'
action = 'use'
event_note='Input use Allen Key event for Lower Frame Assembly'
amount = 1
cur_pros = process_data[f'{process_name}']
cur_res = res_data['Allen_Key_res']

#####
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.03


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Second Pillar
# Pillar_2
# Input event 5
# Define input event work of the Pillar Pillar_2
process_name = 'Second_Pillar_2'
action = 'work'
event_note='Insert three M6nutplates more at the other end of the beam and use one of them to connect another RAconnector there.'
# amount = 0.05
cur_pros = process_data[f'{process_name}']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['assembly_construction']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['assembly_construction']['id']
effort_spec['amount'] = 0.05

event_id, ts = create_event(users_data['one'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# PROCESS wrap
event_seq.append({'ts': ts, 'event_id':cur_pros['id'], 'action' : cur_pros['name'], 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# Second Pillar
# Pillar_2
# Output event 1
# Define output event modify Pillar
process_name = 'Second_Pillar_2'
action = 'modify'
event_note='modify Pillar'
amount = 1
cur_pros = process_data[f'{process_name}']
cur_res = res_data[f'{produced_resource}_res']


event_id, ts = create_event(users_data['two'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Mock Transfer from user two to user one
note='Mock Transfer UpperFrame from two to one'
action = 'transfer'
amount = 1
cur_res = res_data['upper_frame_res']

event_id, ts = make_transfer(users_data['two'], action, note, users_data['one'], amount, cur_res, locs_data, res_spec_data, endpoint=ENDPOINT)
event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
show_data(users_data, locs_data, res_data, units_data, res_spec_data, process_data, event_seq)

In [ ]:
# example of using show functions
# show_resource(users_data['one'], '061Z9FX8M4BJ3M6JMB7MW7VSER', endpoint=ENDPOINT)
# show_proposal(users_data['one'], '061Z9FX9H8PT0JF16KP25VS8R0', endpoint=ENDPOINT)


In [ ]:
trace_me = res_data['upper_frame_res']['id']
# trace_me = '062HCB36KR2QRY5HPP2RETC9M4'
print(f"Resource to be traced: {trace_me}")
tot_dpp = []
visited = {}
er_before(trace_me, users_data['two'], dpp_children=tot_dpp, depth=0, visited=visited, endpoint=ENDPOINT)

# Serializing json
json_object = json.dumps(tot_dpp, indent=2)

print(json_object)
print(visited)

In [ ]:
be_dpp = get_ddp(trace_me, user_data, endpoint=ENDPOINT)
print(json.dumps(be_dpp, indent=2))

In [ ]:
trace = trace_query(trace_me, users_data['two'], endpoint=ENDPOINT)
# check consistency between the registered events, the back-end trace and the generated dpp
check_traces(trace, event_seq, tot_dpp, be_dpp)

In [ ]:
save_traces(USE_CASE, tot_dpp, trace, be_dpp, event_seq)

In [ ]:
labels = []
sources = []
targets = []
values = []
color_nodes = []
color_links = []
assigned = {}
vis_dpp(tot_dpp[0], count=0, assigned=assigned, labels=labels, targets=targets, sources=sources, values=values, color_nodes=color_nodes, color_links=color_links)
sources, targets = consol_trace(assigned, sources, targets)
make_sankey(sources, targets, labels, values, color_nodes, color_links)
# make_sankey([0,0,1,2,2], [2,3,3,3,4], ['0','1','2','3','4'], [2,1,1,1,1], color_nodes, color_links)

In [ ]:
trace_me = res_data['lower_frame_res']['id']
# trace_me = '062HCB36KR2QRY5HPP2RETC9M4'
print(f"Resource to be traced: {trace_me}")
tot_dpp = []
visited = {}
er_before(trace_me, users_data['two'], dpp_children=tot_dpp, depth=0, visited=visited, endpoint=ENDPOINT)

# Serializing json
json_object = json.dumps(tot_dpp, indent=2)

print(json_object)
print(visited)

In [ ]:
be_dpp = get_ddp(trace_me, user_data, endpoint=ENDPOINT)
print(json.dumps(be_dpp, indent=2))

In [ ]:
trace = trace_query(trace_me, users_data['two'], endpoint=ENDPOINT)
# check consistency between the registered events, the back-end trace and the generated dpp
check_traces(trace, event_seq, tot_dpp, be_dpp)

In [ ]:
save_traces(USE_CASE, tot_dpp, trace, be_dpp, event_seq)

In [ ]:
labels = []
sources = []
targets = []
values = []
color_nodes = []
color_links = []
assigned = {}
vis_dpp(tot_dpp[0], count=0, assigned=assigned, labels=labels, targets=targets, sources=sources, values=values, color_nodes=color_nodes, color_links=color_links)
sources, targets = consol_trace(assigned, sources, targets)
make_sankey(sources, targets, labels, values, color_nodes, color_links)
# make_sankey([0,0,1,2,2], [2,3,3,3,4], ['0','1','2','3','4'], [2,1,1,1,1], color_nodes, color_links)